In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [1]:
import pickle
import time
import cknn
import torch
import gc

import numpy as np
import pandas as pd
import DataFormatterEditAddSession as DataFormatterA
import DataFormatterEdit as DataFormatter

from model import *
from tqdm.notebook import tqdm
from utils import build_graph, split_validation, Data

In [2]:
#import importlib
#importlib.reload()
#from IPython.display import display

In [3]:
def num_data(data, str_):
    values = data.loc[:,[str_]].values
    values = values.reshape(-1,)
    new = np.unique(values)
    return new.shape[0]

In [4]:
fold = 5
#_path = 'amz_data_prep_2018_book.csv'
_path = 'amz_data_prep_2018_book_40k.csv'

In [5]:
def make_pd_data_frame(train_data, session_key='session_id', item_key='item_id', time_key='eventdate'):
    df = pd.DataFrame({session_key:[], item_key:[], time_key:[]})
    ses_ids = []
    ses_dates = []
    ses_seqs = []
    for ses_id, ses_date, ses_seq in tqdm(zip(train_data[0], train_data[1], train_data[2]), total=len(train_data[0])):
        for ses_ in ses_seq:
            ses_date_ = int(ses_date.astype(np.float))
            ses_ids.append(int(ses_id))
            ses_dates.append(ses_date_)
            ses_seqs.append(ses_)
    ses_ids = np.asarray(ses_ids)
    ses_dates = np.asarray(ses_dates)
    ses_seqs = np.asarray(ses_seqs)
    d = np.concatenate((ses_ids,ses_seqs,ses_dates))
    d = np.reshape(d,(3,-1))
    d = d.T
    column_names = [session_key, item_key, time_key]
    df = pd.DataFrame(d, columns=column_names)
    return df

In [6]:
data = pd.read_csv(_path, delimiter=',')
n_item = num_data(data, 'item_id')
k_split_data_list, sess_clicks = DataFormatterA.get_k(_path, fold)
last_sr_gnn_hit, sr_gnn_hit, iknn_hit, final_hit, last_sr_gnn_mrr, sr_gnn_mrr, iknn_mrr, final_mrr = [], [], [], [], [], [], [], []
for __p in range(5):
    avg_p = []
    avg_m = []
    for fold_ in range(fold):
        gc.collect()
        test_data_preproc = k_split_data_list[fold_]
        train_data_preproc = k_split_data_list.copy()
        del train_data_preproc[fold_]
        train_data_ = _ = train_data = test_data = df = knn = mrr_20 = p_20 = scr = scr_convd = slices = None

        train_data_preproc, test_data_preproc, train_data_, _ = DataFormatterA.make_tr_ts_data_set(train_data_preproc, 
                                                                                                      test_data_preproc, 
                                                                                                      sess_clicks)

        train_data = Data(train_data_preproc, shuffle=True)
        test_data = Data(test_data_preproc, shuffle=False)
        #test_data.inputs.shape
        #train_data_preproc = [[subset],[ans_items]]
        #train_data_ = [[session_id],[session_date],[item_seq]]
        #train_data = obj
        n_node = n_item+1

        model = trans_to_cuda(SessionGraph(n_node))

        best_result = [0, 0]
        best_epoch = [0, 0]
        bad_counter = 0

        for epoch in range(20):
            hit, mrr = train_test(model, train_data, test_data)
            flag = 0
            if hit > best_result[0]:
                best_result[0] = hit
                best_epoch[0] = epoch
                flag = 1
            if mrr > best_result[1]:
                best_result[1] = mrr
                best_epoch[1] = epoch
                flag = 1
            bad_counter += 1 - flag
            if bad_counter >= 5:
                break
        print('-------------------------------------------------------')
        last_sr_gnn_hit.append(hit)
        last_sr_gnn_mrr.append(mrr)

        df = make_pd_data_frame(train_data_)
        knn = cknn.ContextKNN(data = df, session_key='session_id', item_key='item_id', time_key='eventdate')

        mrr_20, p_20, scr = knn.multi_predict_ts_data(test_data, 12)


        model.eval()
        hit, mrr = [], []
        slices = test_data.generate_batch(model.batch_size)

        for i in tqdm(slices):
            targets, scores = forward(model, i, test_data)
            scores = trans_to_cpu(scores).detach().numpy()
            for j, idx in enumerate(i):
                v = scr[idx][0]
                for index, val in zip(list(v.keys()), list(v.values())):
                    scores[j,index-1] += val
            scores = torch.Tensor(scores)
            sub_scores = scores.topk(20)[1]
            sub_scores = trans_to_cpu(sub_scores).detach().numpy()
            for score, target, mask in zip(sub_scores, targets, test_data.mask):
                hit.append(np.isin(target - 1, score))
                if len(np.where(score == target - 1)[0]) == 0:
                    mrr.append(0)
                else:
                    mrr.append(1 / (np.where(score == target - 1)[0][0] + 1))

        final_hit.append(np.mean(hit)*100)
        final_mrr.append(np.mean(mrr)*100)

        sr_gnn_hit.append(best_result[0])
        sr_gnn_mrr.append(best_result[1])

        iknn_hit.append(p_20)
        iknn_mrr.append(mrr_20)
    print()
    print('SR-GNN Score: ', np.mean(last_sr_gnn_hit), np.mean(last_sr_gnn_mrr))
    print('SR-GNN Last Score: ', np.mean(sr_gnn_hit), np.mean(sr_gnn_mrr))
    print('CKNN Score: ', np.mean(iknn_hit), np.mean(iknn_mrr))
    print('Proposed Method Score: ',np.mean(final_hit), np.mean(final_mrr))
    avg_p.append(np.mean(final_hit))
    avg_m.append(np.mean(final_mrr))
print('Method 1 Avg Proposed Method Score: ',np.mean(avg_p), np.mean(avg_m))
gc.collect()
for __p in range(5):
    avg_p = []
    avg_m = []
    for fold_ in range(fold):
        gc.collect()
        test_data_preproc = k_split_data_list[fold_]
        train_data_preproc = k_split_data_list.copy()
        del train_data_preproc[fold_]
        train_data_ = _ = train_data = test_data = df = knn = mrr_20 = p_20 = scr = scr_convd = slices = None

        train_data_preproc, test_data_preproc, train_data_, _ = DataFormatterA.make_tr_ts_data_set(train_data_preproc, 
                                                                                                      test_data_preproc, 
                                                                                                      sess_clicks)

        train_data = Data(train_data_preproc, shuffle=True, method=0)
        test_data = Data(test_data_preproc, shuffle=False)
        #test_data.inputs.shape
        #train_data_preproc = [[subset],[ans_items]]
        #train_data_ = [[session_id],[session_date],[item_seq]]
        #train_data = obj
        n_node = n_item+1

        model = trans_to_cuda(SessionGraph(n_node))

        best_result = [0, 0]
        best_epoch = [0, 0]
        bad_counter = 0

        for epoch in range(20):
            hit, mrr = train_test(model, train_data, test_data)
            flag = 0
            if hit > best_result[0]:
                best_result[0] = hit
                best_epoch[0] = epoch
                flag = 1
            if mrr > best_result[1]:
                best_result[1] = mrr
                best_epoch[1] = epoch
                flag = 1
            bad_counter += 1 - flag
            if bad_counter >= 5:
                break
        print('-------------------------------------------------------')
        last_sr_gnn_hit.append(hit)
        last_sr_gnn_mrr.append(mrr)

        df = make_pd_data_frame(train_data_)
        knn = cknn.ContextKNN(data = df, session_key='session_id', item_key='item_id', time_key='eventdate')

        mrr_20, p_20, scr = knn.multi_predict_ts_data(test_data, 12)


        model.eval()
        hit, mrr = [], []
        slices = test_data.generate_batch(model.batch_size)

        for i in tqdm(slices):
            targets, scores = forward(model, i, test_data)
            scores = trans_to_cpu(scores).detach().numpy()
            for j, idx in enumerate(i):
                v = scr[idx][0]
                for index, val in zip(list(v.keys()), list(v.values())):
                    scores[j,index-1] += val
            scores = torch.Tensor(scores)
            sub_scores = scores.topk(20)[1]
            sub_scores = trans_to_cpu(sub_scores).detach().numpy()
            for score, target, mask in zip(sub_scores, targets, test_data.mask):
                hit.append(np.isin(target - 1, score))
                if len(np.where(score == target - 1)[0]) == 0:
                    mrr.append(0)
                else:
                    mrr.append(1 / (np.where(score == target - 1)[0][0] + 1))

        final_hit.append(np.mean(hit)*100)
        final_mrr.append(np.mean(mrr)*100)

        sr_gnn_hit.append(best_result[0])
        sr_gnn_mrr.append(best_result[1])

        iknn_hit.append(p_20)
        iknn_mrr.append(mrr_20)
    print()
    print('SR-GNN Score: ', np.mean(last_sr_gnn_hit), np.mean(last_sr_gnn_mrr))
    print('SR-GNN Last Score: ', np.mean(sr_gnn_hit), np.mean(sr_gnn_mrr))
    print('CKNN Score: ', np.mean(iknn_hit), np.mean(iknn_mrr))
    print('Proposed Method Score: ',np.mean(final_hit), np.mean(final_mrr))
    avg_p.append(np.mean(final_hit))
    avg_m.append(np.mean(final_mrr))
print('Method 2 Avg Proposed Method Score: ',np.mean(avg_p), np.mean(avg_m))
gc.collect()
for __p in range(5):
    avg_p = []
    avg_m = []
    for fold_ in range(fold):
        gc.collect()
        test_data_preproc = k_split_data_list[fold_]
        train_data_preproc = k_split_data_list.copy()
        del train_data_preproc[fold_]
        train_data_ = _ = train_data = test_data = df = knn = mrr_20 = p_20 = scr = scr_convd = slices = None

        train_data_preproc, test_data_preproc, train_data_, _ = DataFormatterA.make_tr_ts_data_set(train_data_preproc, 
                                                                                                      test_data_preproc, 
                                                                                                      sess_clicks)

        train_data = Data(train_data_preproc, shuffle=True, method=1)
        test_data = Data(test_data_preproc, shuffle=False)
        #test_data.inputs.shape
        #train_data_preproc = [[subset],[ans_items]]
        #train_data_ = [[session_id],[session_date],[item_seq]]
        #train_data = obj
        n_node = n_item+1

        model = trans_to_cuda(SessionGraph(n_node))

        best_result = [0, 0]
        best_epoch = [0, 0]
        bad_counter = 0

        for epoch in range(20):
            hit, mrr = train_test(model, train_data, test_data)
            flag = 0
            if hit > best_result[0]:
                best_result[0] = hit
                best_epoch[0] = epoch
                flag = 1
            if mrr > best_result[1]:
                best_result[1] = mrr
                best_epoch[1] = epoch
                flag = 1
            bad_counter += 1 - flag
            if bad_counter >= 5:
                break
        print('-------------------------------------------------------')
        last_sr_gnn_hit.append(hit)
        last_sr_gnn_mrr.append(mrr)

        df = make_pd_data_frame(train_data_)
        knn = cknn.ContextKNN(data = df, session_key='session_id', item_key='item_id', time_key='eventdate')

        mrr_20, p_20, scr = knn.multi_predict_ts_data(test_data, 12)


        model.eval()
        hit, mrr = [], []
        slices = test_data.generate_batch(model.batch_size)

        for i in tqdm(slices):
            targets, scores = forward(model, i, test_data)
            scores = trans_to_cpu(scores).detach().numpy()
            for j, idx in enumerate(i):
                v = scr[idx][0]
                for index, val in zip(list(v.keys()), list(v.values())):
                    scores[j,index-1] += val
            scores = torch.Tensor(scores)
            sub_scores = scores.topk(20)[1]
            sub_scores = trans_to_cpu(sub_scores).detach().numpy()
            for score, target, mask in zip(sub_scores, targets, test_data.mask):
                hit.append(np.isin(target - 1, score))
                if len(np.where(score == target - 1)[0]) == 0:
                    mrr.append(0)
                else:
                    mrr.append(1 / (np.where(score == target - 1)[0][0] + 1))

        final_hit.append(np.mean(hit)*100)
        final_mrr.append(np.mean(mrr)*100)

        sr_gnn_hit.append(best_result[0])
        sr_gnn_mrr.append(best_result[1])

        iknn_hit.append(p_20)
        iknn_mrr.append(mrr_20)
    print()
    print('SR-GNN Score: ', np.mean(last_sr_gnn_hit), np.mean(last_sr_gnn_mrr))
    print('SR-GNN Last Score: ', np.mean(sr_gnn_hit), np.mean(sr_gnn_mrr))
    print('CKNN Score: ', np.mean(iknn_hit), np.mean(iknn_mrr))
    print('Proposed Method Score: ',np.mean(final_hit), np.mean(final_mrr))
    avg_p.append(np.mean(final_hit))
    avg_m.append(np.mean(final_mrr))
print('Method 3 Avg Proposed Method Score: ',np.mean(avg_p), np.mean(avg_m))

100%|██████████| 560/560 [00:00<00:00, 10248.85it/s]
/home/workplace/SR-GNN/final/model.py:18: UserWarning: Legacy tensor constructor is deprecated. Use: torch.tensor(...) for creating tensors from tensor-like objects; or torch.empty(...) for creating an uninitialized tensor with specific sizes. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_new.cpp:478.)
  self.w_ih = Parameter(torch.Tensor(self.gate_size, self.input_size))
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
  0%|   

	Loss:	700.828


  0%|          | 0/91 [00:00<?, ?it/s]

	Loss:	471.705


  0%|          | 0/91 [00:00<?, ?it/s]

...

	Loss:	397.905


  0%|          | 0/91 [00:00<?, ?it/s]

	Loss:	397.664


 53%|█████▎    | 48/91 [00:02<00:02, 21.17it/s]


KeyboardInterrupt: 

In [7]:
print('done')

done


SR-GNN Score:  49.338173340459726 39.97791667795731
SR-GNN Last Score:  49.48027060489741 39.98477209029336
CKNN Score:  40.071607630996915 24.28932932064422
Proposed Method Score:  54.88065544799984 40.69128085092071
SR-GNN Score:  48.82411896393794 39.76567653644829
SR-GNN Last Score:  48.89516759615678 39.77259106514005
CKNN Score:  40.07160763099692 24.289329320644217
Proposed Method Score:  54.5434932542232 40.48296437244213
SR-GNN Score:  49.03070046783436 39.84645586608671
SR-GNN Last Score:  49.148988208462534 39.85774601779054
CKNN Score:  40.07160763099692 24.28932932064422
Proposed Method Score:  54.585128428000964 40.53196971328937
SR-GNN Score:  48.96572538920712 39.9082234640969
SR-GNN Last Score:  49.08122690896396 39.916720049274204
CKNN Score:  40.07160763099692 24.289329320644217
Proposed Method Score:  54.59708009181527 40.59958214407139
SR-GNN Score:  48.94107644493874 39.838279073837725
SR-GNN Last Score:  49.03347766074421 39.845204623013714
CKNN Score:  40.07160763099692 24.289329320644214
Proposed Method Score:  54.51918303846992 40.54599754062751
Method 1 Avg Proposed Method Score:  54.51918303846992 40.54599754062751

SR-GNN Score:  48.11847103655715 39.2057053415415
SR-GNN Last Score:  48.15412166044307 39.20580884513826
CKNN Score:  44.816038164553774 32.47939661724543
Proposed Method Score:  53.939598209922906 41.098498802422924
SR-GNN Score:  47.728084625901445 38.848617854854226
SR-GNN Last Score:  47.745909937844395 38.84875388815281
CKNN Score:  44.81603816455377 32.47939661724543
Proposed Method Score:  53.708689658656866 41.038768206549165
SR-GNN Score:  48.24889650150303 38.88524991286906
SR-GNN Last Score:  48.26078004279834 38.885435547942976
CKNN Score:  44.81603816455377 32.47939661724543
Proposed Method Score:  53.89202085875224 40.73011427902729
SR-GNN Score:  48.49108362989154 39.02498285116509
SR-GNN Last Score:  48.53545761440087 39.02659248494556
CKNN Score:  44.816038164553774 32.47939661724543
Proposed Method Score:  53.921393354940996 40.63162025548813
SR-GNN Score:  48.72189526857687 39.050454661048
SR-GNN Last Score:  48.757394456184336 39.05186666855066
CKNN Score:  44.81603816455377 32.47939661724543
Proposed Method Score:  53.98861270420677 40.54752250153825
Method 1 Avg Proposed Method Score:  53.98861270420677 40.54752250153825

SR-GNN Score:  48.52702499350679 39.106945040227394
SR-GNN Last Score:  48.56841752034572 39.112912183746786
CKNN Score:  44.81603816455377 32.47939661724543
Proposed Method Score:  53.98045400718165 40.60424014995526
SR-GNN Score:  48.63624919077486 39.212073338877154
SR-GNN Last Score:  48.671728499493945 39.21761018799983
CKNN Score:  44.816038164553774 32.47939661724543
Proposed Method Score:  54.08103659388731 40.60352193671038
SR-GNN Score:  48.55870265102725 39.19346697201373
SR-GNN Last Score:  48.59417182491751 39.198353745237206
CKNN Score:  44.81603816455377 32.47939661724543
Proposed Method Score:  54.063356711686176 40.61730094980477
SR-GNN Score:  48.533611894126516 39.12721319877099
SR-GNN Last Score:  48.58091451160068 39.13155699719186
CKNN Score:  44.81603816455377 32.47939661724543
Proposed Method Score:  54.04176053849841 40.545403776459835
SR-GNN Score:  48.58447411683145 39.1367224817727
SR-GNN Last Score:  48.63766594158474 39.14133812332717
CKNN Score:  44.81603816455377 32.47939661724543
Proposed Method Score:  54.063434407225905 40.54142512509033
Method 2 Avg Proposed Method Score:  54.063434407225905 40.54142512509033

SR-GNN Score:  48.590765339984316 39.12659448086034
SR-GNN Last Score:  48.642339565222635 39.130790518637134
CKNN Score:  44.81603816455376 32.47939661724543
Proposed Method Score:  54.05216522092618 40.52465604434711
SR-GNN Score:  48.5604520681143 39.14678786988186
SR-GNN Last Score:  48.61068352399174 39.15063423784391
CKNN Score:  44.81603816455377 32.47939661724543
Proposed Method Score:  54.05469440346543 40.53582844787609
SR-GNN Score:  48.567676866887645 39.13613415565544
SR-GNN Last Score:  48.61677213330588 39.13968464915888
CKNN Score:  44.81603816455376 32.47939661724542
Proposed Method Score:  54.10325061945461 40.541453194420896
SR-GNN Score:  48.48246242571709 39.17214756440152
SR-GNN Last Score:  48.5331257414891 39.17544673972324
CKNN Score:  44.81603816455376 32.47939661724543
Proposed Method Score:  54.07893248166918 40.59627225120298
SR-GNN Score:  48.48429417220811 39.18395485627738
SR-GNN Last Score:  48.53394399978918 39.187034086577654
CKNN Score:  44.81603816455377 32.47939661724543
Proposed Method Score:  54.06491956683783 40.58811045570778
Method 3 Avg Proposed Method Score:  54.06491956683783 40.58811045570778

40k
CKNN Score:  44.81603816455377 32.47939661724543

SR-GNN Last Score:    48.29073274233419 39.003691486946046
Method 1 Avg Proposed Method Score:  53.98861270420677 40.54752250153825

SR-GNN Last Score:   48.610579659588524 39.16035424750057
Method 2 Avg Proposed Method Score:  54.063434407225905 40.54142512509033

SR-GNN Last Score:   48.58737299275971 39.156718046388164
Method 3 Avg Proposed Method Score:  54.06491956683783 40.58811045570778

2018
CKNN Score:  8.965137202665257 5.687222376937899

SR-GNN Last Score:  24.12804519933014 14.705060071171676
Method 1 Avg Proposed Method Score:  24.513084956296794 12.923072506624601

SR-GNN Last Score:  24.30988855051716 14.588509508467718
Method 2 Avg Proposed Method Score:  24.67480070201331 12.747575408266078

SR-GNN Last Score:  24.111452628616544 14.477994448393519
Method 3 Avg Proposed Method Score:  24.54353790525956 12.757424840740866

40k
data: 40000
user: 9110
item: 21874
avg_item: 4.390779363336993

2018
data: 2467321
user: 1361089
item: 492767
avg_item: 1.8127550806743717